In [ ]:
''' IMPORTS '''

import pandas as pd
import numpy as np
from scipy import stats
import math
import requests
from io import BytesIO
from datetime import datetime

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl
from plotly.subplots import make_subplots

from PIL import Image

from resources.tier_chart import tier_chart
from resources.plotly_theme import nfl_template
from resources.get_nfl_data import get_team_info, get_pbp_data
from resources.team_stats import get_team_stats

pio.templates['nfl_template'] = nfl_template

In [ ]:
''' Params / Constants '''

EXPORT = False

## Params ##
PREVIEW_WEEK = 15

AWAY_TEAM = 'IND'
HOME_TEAM = 'SEA'
MATCHUP_TEAMS = [AWAY_TEAM, HOME_TEAM]

## Constants ##

LEAGUE_LOGO = 'https://raw.githubusercontent.com/nflverse/nflverse-pbp/master/NFL.png'
PLOT_BUFFER = 0.1

In [ ]:
''' Import Data '''

# Import
team_data = get_team_info().rename_axis(index={'team_abbr': 'team'})
pbp_data = get_pbp_data(years=[2025])

player_info = nfl.import_players()

# PBP
league_data = pbp_data.loc[(pbp_data['week'] < PREVIEW_WEEK),:]
pbp_matchup_teams = league_data.loc[(league_data['posteam'].isin(MATCHUP_TEAMS)), :].copy()

# Stats
team_offense = get_team_stats(pbp_data, unit='offense')
team_offense = team_offense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)
print(team_offense.sort_values(by='EPA / Play', ascending=False).head().to_string())

WORDMARKS = [team_data.loc[team_data.index == AWAY_TEAM, 'team_wordmark'].values[0], team_data.loc[team_data.index == HOME_TEAM, 'team_wordmark'].values[0]]
LOGOS = [team_data.loc[team_data.index == AWAY_TEAM, 'team_logo_espn'].values[0], team_data.loc[team_data.index == HOME_TEAM, 'team_logo_espn'].values[0]]
COLORS = [team_data.loc[team_data.index == AWAY_TEAM, 'team_color'].values[0], team_data.loc[team_data.index == HOME_TEAM, 'team_color'].values[0]]

print(team_data.shape)
print(team_data.head().to_string())
print(pbp_matchup_teams.shape)
print(pbp_matchup_teams.head().to_string())

In [ ]:
print(player_info.head().to_string())

# Team Form

In [ ]:
league_adv_offense_pbp = league_data.loc[(league_data['Offensive Snap']) &
                                         (~league_data['Is Special Teams Play']), :].copy()

league_av_epa = league_adv_offense_pbp['epa'].mean()
print(f'League av EPA / Play: {league_av_epa:,.2f}')

for TEAM in MATCHUP_TEAMS:
    for unit in ['posteam', 'defteam']:
        opp_unit = 'defteam' if unit == 'posteam' else 'posteam'
        # Wrangle
        team_sl = league_adv_offense_pbp.loc[league_adv_offense_pbp[unit] == TEAM, ['game_id', 'posteam', 'defteam', 'epa']].copy()
        team_sl['Rolling EPA / Play'] = team_sl.rolling(window=75, closed='left')['epa'].mean()

        team_sl = team_sl.merge(team_data[['team_color', 'team_logo_espn']], 
                            left_on=opp_unit, right_index=True, how='left').rename(columns={
                                'team_color': 'opp_color',
                                'team_logo_espn': 'opp_logo'
                            })

        team_sl = team_sl.reset_index(drop=True)
        team_sl.index = team_sl.index + 1
        # print(team_sl.head().to_string())

        # Data
        x = team_sl.index
        y = team_sl['Rolling EPA / Play'].to_numpy()
        opp = team_sl[opp_unit].to_numpy()
        colors = team_sl['opp_color'].tolist()
        color_map = team_data['team_color'].to_dict()
        team_wordmark = team_data.loc[TEAM, 'team_wordmark']

        # Games
        games = team_sl['game_id'].unique().tolist()
        opp_logos = team_sl.drop_duplicates(subset='game_id')['opp_logo'].tolist()
        opp_colors = team_sl.drop_duplicates(subset='game_id')['opp_color'].tolist()
        game_endpoints = []
        game_midpoints = []
        for g in games:
            sl = team_sl.loc[team_sl['game_id'] == g, :]
            midpoint_play = (sl.index[-1] + sl.index[0]) / 2
            game_midpoints.append(midpoint_play)
            game_endpoints.append((sl.index[0],sl.index[-1]))

        # Figure
        fig = px.line(
            data_frame=team_sl,
            x=x,
            y=y,
            color_discrete_sequence=['#323232']
        )

        fig.add_hline(y=league_av_epa, line_width=1, line_dash="dash", line_color="#323232", layer='above',
                      annotation=dict(text='League avg', font=dict(color='white', size=10), yanchor='bottom', xanchor='left'), annotation_position='left')

        # Opp logo
        for i in range(len(games)):
            fig.add_layout_image(
                x=game_midpoints[i],
                y=1,
                sizex=team_sl.index[-1]*.04,
                sizey=team_sl.index[-1]*.04,
                xanchor='center',
                yanchor='middle',
                xref='x', 
                yref='paper',
                source=opp_logos[i],
            )
            fig.add_shape(
                type="rect",
                xref='x',
                x0=game_endpoints[i][0],  # x-coordinate of the left edge
                x1=game_endpoints[i][1],  # x-coordinate of the right edge
                yref='paper',
                y0=0,  # y-coordinate of the bottom edge
                y1=1,  # y-coordinate of the top edge
                fillcolor=opp_colors[i],  # Color to fill the rectangle
                opacity=0.6,  # Opacity of the fill color
                line=dict(color="white", width=1),  # Line properties for the border
                layer="below"  # Place the shape below the traces
            )
            fig.add_annotation(
                text=f'{game_endpoints[i][1] - game_endpoints[i][0]} Plays',
                font=dict(color='white', size=10),
                xref='x', 
                yref='paper',
                x=game_midpoints[i],
                y=.975,
                align='center',
                showarrow=False
            )

        # Team wordmark
        response = requests.get(team_wordmark)
        logo_img = Image.open(BytesIO(response.content))
        fig.add_layout_image(
            x=0.5,
            y=1.1,
            sizex=.15,
            sizey=.15,
            xanchor='center',
            yanchor='middle',
            xref='paper', 
            yref='paper',
            source=logo_img,
        )

        fig.update_traces(line=dict(width=3))
        fig.update_yaxes(
            linecolor='#f0f0f0', mirror=True,
            title='Rolling EPA / Play',
            tickformat='.2f',
            title_standoff=1,
            range=[-0.6, 0.6] if unit == 'posteam' else [0.6, -0.6],
        )
        fig.update_xaxes(
            linecolor='#f0f0f0', mirror=True,
            title='Play #',
            tickformat=',',
            showgrid=False,
            title_standoff=1
        )
        title = f'Team Form: {"Offense" if unit == "posteam" else "Defense"}'
        fig.update_layout(
            title=f'<b>{title}</b><br><sup>75-play rolling EPA / Play</sup>',
            template='nfl_template',
            showlegend=False,
            margin=dict(t=75, b=40)
        )
        # Credits
        fig.add_annotation(
            text=f'Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
            showarrow=False,
            xref='paper',
            yref='paper',
            y=-0.09, 
            x=1,
            align='right'
        )

        fig.show()

        if EXPORT: pio.write_image(fig, f'{title} - {TEAM} - Week {PREVIEW_WEEK} - {AWAY_TEAM} vs. {HOME_TEAM}.png',
                                   scale=6, width=900, height=500)

# Tendencies

In [ ]:
## Normal Game State

# Filter 1st downs to only 1st and 10
league_pbp_normal_gs = league_data.loc[~((league_data['down'] == 1) & (league_data['ydstogo'] != 10)),:].copy()
league_pbp_normal_gs['Down & Distance'] = league_pbp_normal_gs['Down & Distance'].str.replace('1st & Long', '1st & 10')

# Only normal snaps, Q1-3 and within 2 tds
league_pbp_normal_gs = league_pbp_normal_gs.loc[(~league_pbp_normal_gs['Is Special Teams Play']) &
                            (league_pbp_normal_gs['qtr'] <= 3) & 
                            (league_pbp_normal_gs['score_differential'] <= 14),:]

#
league_pbp_normal_gs['Is 1st Down'] = league_pbp_normal_gs['down'] == 1


In [ ]:
''' Overall '''

overall_gpby = league_pbp_normal_gs.groupby(['posteam']).aggregate(
    Plays=('Offensive Snap', 'sum'),
    Pass=('pass', 'sum'),
    Rush=('rush', 'sum'),
).rename_axis(index={'posteam': 'team'})
overall_gpby['% Pass'] = overall_gpby['Pass'] / overall_gpby['Plays']
overall_gpby['% Rush'] = overall_gpby['Rush'] / overall_gpby['Plays']

print(overall_gpby.head().to_string())


In [ ]:
''' Pass / Rushing by Down & Distance '''


## Groupby
down_distance_gpby = league_pbp_normal_gs.groupby(['posteam', 'down', 'Down & Distance']).aggregate(
    Plays=('Offensive Snap', 'sum'),
    Pass=('pass', 'sum'),
    Rush=('rush', 'sum'),
).rename_axis(index={'posteam': 'team'})

sub_totals_1 = down_distance_gpby.groupby(['team', 'down']).sum()
sub_totals_1['Down & Distance'] = 'Total'
sub_totals_1 = sub_totals_1.set_index('Down & Distance', append=True)

sub_totals_2 = down_distance_gpby.groupby('team').sum()
sub_totals_2['down'] = 'Total'
sub_totals_2['Down & Distance'] = 'Total'
sub_totals_2 = sub_totals_2.set_index('down', append=True)
sub_totals_2 = sub_totals_2.set_index('Down & Distance', append=True)

down_distance_gpby = pd.concat([down_distance_gpby, sub_totals_1, sub_totals_2]).sort_index()
down_distance_gpby['% Pass'] = down_distance_gpby['Pass'] / down_distance_gpby['Plays']
down_distance_gpby['% Rush'] = down_distance_gpby['Rush'] / down_distance_gpby['Plays']
down_distance_gpby['Diff'] = down_distance_gpby['% Pass'] - down_distance_gpby['% Rush']

down_distance_gpby = down_distance_gpby.merge(team_data['team_logo_espn'], left_index=True, right_index=True)
print(down_distance_gpby.head().to_string())

## Figure

# Data
data = down_distance_gpby.loc[(down_distance_gpby.index.get_level_values('down') != 4) &
                              (down_distance_gpby.index.get_level_values(2) != 'Total'), :]

x = data['% Pass'].tolist()
y = data.index.get_level_values(2).tolist()
logos = data['team_logo_espn'].tolist()
teams = data.index.get_level_values(0).tolist()

y_order = ['1st & 10']
for down in ['2nd', '3rd']:
    for distance in ['Short', 'Medium', 'Long']:
        y_order.append(f'{down} & {distance}')

# Plot
dot_plot = px.scatter(
    x=x,
    y=y,
)

# Init figure
fig = go.Figure()

for trace in dot_plot.data:
    fig.add_trace(trace)

# Logos
for i in range(len(logos)):
    if math.isnan(x[i]): 
        continue
    op = 1 if teams[i] in MATCHUP_TEAMS else 0.4
    size = 0.6 if teams[i] in MATCHUP_TEAMS else 0.4
    layer = 'above' if teams[i] in MATCHUP_TEAMS else 'below'
    fig.add_layout_image(
        source=logos[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=x[i], # X-coordinate of the image's center
        y=y[i], # Y-coordinate of the image's center
        sizex=size,   # Width of the image in data units
        sizey=size,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer=layer, # Place image above other plot elements
        opacity=op
    )

# Format
fig.update_traces(marker=dict(opacity=0))

fig.update_xaxes(
    title=dict(
        text=f'<span style="font-size: 10px"><-- More Run Heavy</span>       <b>% Pass</b>       <span style="font-size: 10px">More Pass Heavy --></span>',
        font=dict(weight='normal')
    ),
    tickformat='.0%',
    range=[-0.05, 1.05],
    dtick=.1,
    linecolor='#f0f0f0', mirror=True,
)
fig.update_yaxes(
    categoryorder="array", 
    categoryarray=y_order,
    autorange='reversed',
    linecolor='#f0f0f0', mirror=True,
    showgrid=False,
)
fig.update_layout(
    template='nfl_template',
    title=f'<b>Pass / Rush Tendencies by Down & Distance</b><br><sup>"Normal" game state: qtrs 1-3, score within 14 pts</sup>',
    margin=dict(t=50, l=75, b=60),
)

# Credits
fig.add_annotation(
    text=f'Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.125, 
    x=1,
    align='right'
)

fig.show()

if EXPORT: pio.write_image(fig, f'Pass Rush Tendencies by Down & Distance - Week {PREVIEW_WEEK} - {AWAY_TEAM} vs {HOME_TEAM}.png',
                           scale=6, width=900, height=500)

In [ ]:
league_data['run loc full'] = np.where(league_data['run_location'] == 'middle', 'middle', league_data['run_location'] + ' ' + league_data['run_gap'])

def pass_length(air_yards):
    if not air_yards:
        return
    
    if air_yards <= 10:
        return 'Short'
    elif air_yards <= 20:
        return 'Medium'
    else:
        return 'Long'
    
league_data['pass length'] = league_data['air_yards'].apply(lambda x: pass_length(x))
league_data['pass loc full'] = league_data['pass length'] + ' ' + league_data['pass_location']

In [ ]:
print(league_data['pass loc full'].value_counts())
print(league_data['pass loc full'].unique().tolist())
print(league_data.head().to_string())

In [ ]:
''' Pass Locations '''

pass_loc_order = ['Short left', 'Short middle', 'Short right', 'Medium left', 'Medium middle', 'Medium right', 'Long left', 'Long middle', 'Long right']

# Team
by_pass_loc = league_data.groupby(['posteam', 'pass length', 'pass_location']).aggregate(
    Plays=('pass', 'sum'),
    Yards=('passing_yards', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum'),
    EPA=('epa', 'sum')
)
by_pass_loc['% Plays'] = by_pass_loc['Plays'] / by_pass_loc.groupby(level=0)['Plays'].sum()
by_pass_loc['Success Rate'] = by_pass_loc['Successes'] / by_pass_loc['Plays']
by_pass_loc['EPA / Play'] = by_pass_loc['EPA'] / by_pass_loc['Plays']

# Target leaders
pass_loc_targets = league_data.groupby(['posteam', 'receiver', 'pass length', 'pass_location']).aggregate(
    Player_ID=('receiver_player_id', 'first'),
    Plays=('pass', 'sum'),
    Yards=('passing_yards', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum'),
    EPA=('epa', 'sum')
)
pass_loc_targets['% Plays'] = pass_loc_targets['Plays'] / pass_loc_targets.groupby(level=0)['Plays'].sum()
pass_loc_targets['Success Rate'] = pass_loc_targets['Successes'] / pass_loc_targets['Plays']
pass_loc_targets['EPA / Play'] = pass_loc_targets['EPA'] / pass_loc_targets['Plays']

target_leaders = pass_loc_targets.groupby(level=[0,2,3])['Plays'].aggregate(['idxmax', 'max'])
target_leaders['Target Leader'] = target_leaders['idxmax'].str[1]
target_leaders['Targets'] = target_leaders['max']

# Combine
by_pass_loc = by_pass_loc.merge(target_leaders[['Target Leader', 'Targets']], left_index=True, right_index=True)

team_player_ids = pass_loc_targets.groupby(level=[0,1])['Player_ID'].first().reset_index()
team_player_ids = team_player_ids.merge(player_info[['gsis_id', 'headshot']], left_on='Player_ID', right_on='gsis_id', how='left').drop(columns=['gsis_id'])
print(team_player_ids.head().to_string())
by_pass_loc = by_pass_loc.reset_index().merge(team_player_ids, left_on=['posteam', 'Target Leader'], right_on=['posteam', 'receiver'], how='left').set_index(['posteam', 'pass length', 'pass_location'])

by_pass_loc['text'] = 'EPA / Play:<br>' + by_pass_loc['EPA / Play'].round(2).astype(str) + '<br><br>' + 'Target Leader:<br>' + by_pass_loc['Target Leader'] + ' (' + by_pass_loc['Targets'].astype(int).astype(str) + ')'

print(by_pass_loc.loc[by_pass_loc.index.get_level_values(0) == 'IND',:].to_string())

In [ ]:
''' Run Locations '''

run_loc_order = ['left end', 'left tackle', 'left guard', 'middle', 'right guard', 'right tackle', 'right end']

by_run_loc = league_data.groupby(['posteam', 'run loc full']).aggregate(
    Plays=('rush', 'sum'),
    Yards=('rushing_yards', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum'),
    EPA=('epa', 'sum'),
    Stuffs=('rush', lambda x: x[(league_data['rushing_yards'] <= 0)].sum())
)
by_run_loc['% Plays'] = by_run_loc['Plays'] / by_run_loc.groupby(level=0)['Plays'].sum()
by_run_loc['Success Rate'] = by_run_loc['Successes'] / by_run_loc['Plays']
by_run_loc['EPA / Play'] = by_run_loc['EPA'] / by_run_loc['Plays']
by_run_loc['Stuff Rate'] = by_run_loc['Stuffs'] / by_run_loc['Plays']


In [ ]:


## Heat maps
color_min = by_pass_loc['EPA / Play'].min()
color_max = by_pass_loc['EPA / Play'].max()

heat_maps = []
text_arrays = []
for team in MATCHUP_TEAMS:
    team_pass_loc = by_pass_loc.loc[by_pass_loc.index.get_level_values(0) == team, :]
    team_pass_loc = team_pass_loc.reindex(labels=['Short', 'Medium', 'Long'], level='pass length').sort_index(ascending=False, level='pass length')
    team_pass_loc = team_pass_loc.reindex(labels=['left', 'middle', 'right'], level='pass_location')

    print(team_pass_loc.to_string())

    x = team_pass_loc.index.get_level_values('pass_location').to_numpy()
    y = team_pass_loc.index.get_level_values('pass length').to_numpy()
    img = team_pass_loc['EPA / Play'].to_numpy()
    text = team_pass_loc['text'].to_numpy()

    heat_map = go.Heatmap(
        x=x, 
        y=y, 
        z=img,
        coloraxis='coloraxis',
        textfont=dict(size=8),
        xgap=1, ygap=1
    )
    
    heat_maps.append(heat_map)
    text_arrays.append(text)


## Figure ##

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "xy"},{"type": "xy"}]],
                    shared_xaxes=True, shared_yaxes=True,
                    column_widths=[1,1])

for i in range(len(heat_maps)):
    fig.add_trace(
        heat_maps[i],
        row=1, col=1+i
    )

    text = text_arrays[i]
    xref = 'x' if i == 0 else 'x2'
    yref = 'y' if i == 0 else 'y2'
    for i in range(len(text)):
        fig.add_annotation(
            x=x[i],
            y=y[i],
            xref=xref,
            yref=yref,
            text=text[i],
            showarrow=False,
            align='left',
            font=dict(color='#323232', size=10),
        )

fig.update_xaxes(
    showgrid=False,
    scaleanchor='y', scaleratio=1
)
fig.update_yaxes(
    showgrid=False,
    scaleanchor='x', scaleratio=1
)
fig.update_layout(
    template='nfl_template',
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(),
    coloraxis=dict(
        colorbar=dict(
            title='EPA / Play',
        ),
        cmid=league_av_epa,
        colorscale=px.colors.diverging.PRGn
    )
)

fig.show()


In [ ]:
# scatter = px.scatter(
    #     x=ind_sl.index.get_level_values('pass_location').to_numpy(), 
    #     y=ind_sl.index.get_level_values('pass length').to_numpy(), 
    #     color=ind_sl['EPA / Play'].to_numpy(),
    #     size=ind_sl['% Plays'].to_numpy(),
    #     color_continuous_scale=px.colors.diverging.PRGn,
    #     template='nfl_template',
    #     size_max=40
    # )

    # Field
    # img = Image.open('./resources/field los.png')
    # img_width, img_height = img.size
    # ratio = img_width / img_height

    # x = (SIZEX*(c-1)) + (X_SPACING*(c-1))
    # y = 1 - ((SIZEY*(r-1)) + (Y_SPACING*(r-1)))
    # scatter.add_layout_image(
    #     x=0,
    #     y=1,
    #     sizey=1,
    #     sizex=1,
    #     xref="paper",
    #     yref="paper",
    #     # opacity=0.2,
    #     layer="below",
    #     sizing="fill",
    #     source=img
    # )

    # scatter.update_traces(marker=dict(line=dict(width=2, color='#323232')))
    # scatter.update_coloraxes(colorbar=dict(title='EPA / Play'), cmin=color_min, cmax=color_max)
    # scatter.update_xaxes(showgrid=False)
    # scatter.update_yaxes(showgrid=False, autorange='reversed')
    # scatter.update_layout(
    #     plot_bgcolor='#cccccc',
    #     margin=dict(pad=5),
    #     height=500*ratio,
    #     width=700
    # )
    # scatter.show()


In [ ]:
''' QB Locations '''

qb_loc = league_data.loc[league_data['qb_location'] != '0', :].groupby(['posteam', 'qb_location']).aggregate(
    Plays=('qb_location', 'size'),
    Pass=('pass', 'sum'),
    Rush=('rush', 'sum'),
    Yards=('yards_gained', 'sum'),
    PassYards=('passing_yards', 'sum'),
    RushYards=('rushing_yards', 'sum'),
)
qb_loc['% Plays'] = qb_loc['Plays'] / qb_loc.groupby(level=0)['Plays'].transform('sum')

print(qb_loc.head().to_string())

In [ ]:
''' QB Location '''

run_loc_order = ['left end', 'left tackle', 'left guard', 'middle', 'right guard', 'right tackle', 'right end']

## Charts ##
play_types = qb_loc_melt['Play Type'].unique().tolist()
play_type_color_map = {}
for i in range(len(play_types)):
    play_type_color_map[play_types[i]] = color=px.colors.qualitative.T10[i]


pies = []
bars = []
pass_locs = []
pass_loc_texts = []
pass_loc_xs = []
pass_loc_ys = []
pass_loc_headshots = []
run_locs = []

for team in MATCHUP_TEAMS:
    # QB Pos
    qb_loc_sl = qb_loc.loc[qb_loc.index.get_level_values('posteam') == team, :]
    
    qb_loc_pie = go.Pie(
        labels=qb_loc_sl.index.get_level_values('qb_location'),
        values=qb_loc_sl['Plays'],
        hole=0.4,
        opacity=1,
        marker=dict(line=dict(width=2, color='#323232')),
        legend='legend',
        legendgroup='QB Position'
    )

    # Play Types
    qb_loc_melt_sl = qb_loc_melt.loc[qb_loc_melt.index.get_level_values('posteam') == team, :]

    team_bars = []
    for play_type in play_types:
        play_type_sl = qb_loc_melt_sl.loc[qb_loc_melt_sl['Play Type'] == play_type,:]

        x = play_type_sl['% Plays'].to_numpy()
        y = play_type_sl.index.get_level_values('qb_location').to_numpy()
        text=play_type_sl['% Plays'].nlargest(n=2).to_numpy()
        pattern = 'x' if play_type in ['RPO Rush', 'RPO Pass', 'PA Pass'] else ''

        team_bar = go.Bar(
            x=x,
            y=y,
            text=text,
            name=play_type,
            marker=dict(pattern=dict(shape=pattern, size=2), color=play_type_color_map[play_type], line=dict(width=2, color='#323232')),
            legend='legend2',
            legendgroup='Play Type',
            orientation='h',
            opacity=1
        )
        team_bars.append(team_bar)

    # Pass loc
    team_pass_loc = by_pass_loc.loc[by_pass_loc.index.get_level_values(0) == team, :]
    team_pass_loc = team_pass_loc.reindex(labels=['Short', 'Medium', 'Long'], level='pass length').sort_index(ascending=False, level='pass length')
    team_pass_loc = team_pass_loc.reindex(labels=['left', 'middle', 'right'], level='pass_location')

    pass_loc_x = team_pass_loc.index.get_level_values('pass_location').to_numpy()
    pass_loc_y = team_pass_loc.index.get_level_values('pass length').to_numpy()
    pass_loc_z = team_pass_loc['% Plays'].to_numpy()
    pass_loc_text = team_pass_loc['text'].to_numpy()
    headshots = team_pass_loc['headshot'].to_numpy()
    print(pass_loc_z)
    
    pass_loc = go.Heatmap(
        x=x, 
        y=y, 
        z=pass_loc_z,
        coloraxis='coloraxis',
        textfont=dict(size=8),
        xgap=1, ygap=1
    )

    # Run Loc
    run_loc_sl = by_run_loc.loc[by_run_loc.index.get_level_values(0) == team,:]
    run_loc_sl = run_loc_sl.reindex(labels=run_loc_order, level=1)

    x = ['L EDGE', 'LT', 'LG', 'C', 'RG', 'RT', 'R EDGE'] #run_loc_sl.index.get_level_values(1).tolist()
    y = run_loc_sl.index.get_level_values(0).unique().tolist()
    z = run_loc_sl[['% Plays']].transpose().to_numpy()
    text = z
    run_loc = go.Heatmap(
        x=x,
        y=y,
        z=z,
        coloraxis='coloraxis2',
        text=z,
        texttemplate='%{text:.0%}',
        xgap=1, ygap=1
    )

    pies.append(qb_loc_pie)

    bars.append(team_bars)

    pass_locs.append(pass_loc)
    pass_loc_texts.append(pass_loc_text)
    pass_loc_xs.append(pass_loc_x)
    pass_loc_ys.append(pass_loc_y)
    pass_loc_headshots.append(headshots)

    run_locs.append(run_loc)


## Figure ##
H_SPACING = 0.1
V_SPACING = 0.06
N_COLS = 2
N_ROWS = 4
row_titles = ['QB Position', 'Play Types', 'Pass Locs', 'Run Locs']
fig = make_subplots(rows=N_ROWS, cols=N_COLS, 
                    specs=[[{'type': 'domain'}, {'type': 'domain'}], [{'type': 'xy'}, {'type': 'xy'}], [{'type': 'xy'}, {'type': 'xy'}], [{'type': 'xy'}, {'type': 'xy'}]], 
                    horizontal_spacing=H_SPACING, vertical_spacing=V_SPACING,
                    row_titles=row_titles,
                    row_heights=[3,3,5,1],
                    print_grid=True)


# Pies
for i in range(len(pies)):
    fig.add_trace(
        pies[i],
        row=1, col=1+i
    )

fig.update_traces(sort=False, marker=dict(colors=px.colors.qualitative.G10), row=1)

# Bars
for i in range(len(bars)):
    if i == 1:
        for bar in bars[i]:
            bar.update(showlegend=False)

    fig.add_traces(
        data=bars[i],
        rows=2, cols=1+i
    )

fig.update_traces(texttemplate='%{value}', row=2)

# Pass locs
for i in range(len(heat_maps)):
    x = pass_loc_xs[i]
    y = pass_loc_ys[i]
    xref = 'x3' if i == 0 else 'x4'
    yref = 'y3' if i == 0 else 'y4'

    # Heatmap
    fig.add_trace(
        heat_maps[i],
        row=3, col=1+i
    )

    # Text
    text = pass_loc_texts[i]
    for t in range(len(text)):
        fig.add_annotation(
            x=x[t],
            y=y[t],
            xref=xref,
            yref=yref,
            yanchor='middle',
            xanchor='center',
            showarrow=False,
            align='left',
            text=text[t],
            font=dict(color='#323232', size=10)
        )

    # Headshot
    headshots = pass_loc_headshots[i]
    for h in range(2):
        response = requests.get(headshots[h])
        headshot = Image.open(BytesIO(response.content))

        fig.add_layout_image(
            source=headshot,
            xref=xref, 
            yref=yref,
            x=x[h],
            y=y[h],
            sizex=0.5,
            sizey=0.5,
            xanchor='center',
            yanchor='middle',
            opacity=0.9
        )

# Run Locs
for i in range(len(run_locs)):
    # for trace in run_locs[i].data:
    fig.add_trace(
        run_locs[i],
        row=4, col=1+i
    )


# Wordmarks
unit_size = (1 - H_SPACING) / 2
for i in range(len(WORDMARKS)):
    response = requests.get(WORDMARKS[i])
    logo_img = Image.open(BytesIO(response.content))
    fig.add_layout_image(
        x=(unit_size / 2) + ((unit_size+H_SPACING)*i),
        y=1.05,
        sizex=.2,
        sizey=.2,
        xanchor='center',
        yanchor='middle',
        xref='paper', 
        yref='paper',
        source=logo_img,
    )


# Formatting
fig.for_each_annotation(lambda a: a.update(x=-.2, textangle=-90, font=dict(weight='bold', size=14)) if a.text in row_titles else())


fig.update_xaxes(
    linecolor='#f0f0f0', mirror=True,
)
fig.update_yaxes(
    linecolor='#f0f0f0', mirror=True
)
fig.update_xaxes(
    row=2,
    tickformat='.0%',
    range=[-.02, 1.02],
)
fig.update_yaxes(
    row=2,
    autorange='reversed',
)

# By Rows / Cols
fig.update_xaxes(
    col=1,
    showticklabels=True,
)
fig.update_yaxes(
    col=1,
    showticklabels=True,
)
fig.update_xaxes(
    col=2,
    showticklabels=True,
)

fig.update_yaxes(
    col=2,
    showticklabels=False,
)

# Heatmaps formatting
for row in [3,4]:
    fig.update_xaxes(
        row=row,
        showgrid=False,
    )
    fig.update_yaxes(
        row=row,
        showgrid=False,
    )

fig.update_yaxes(
    row=4,
    showticklabels=False
)

width=1000
height=1250
fig.update_layout(
    template='nfl_template',
    title=f'<b>Offensive Tendencies</b><br><sup>Week {PREVIEW_WEEK}: {AWAY_TEAM} vs. {HOME_TEAM}</sup>',
    barmode='stack',
    coloraxis=dict(
        colorbar=dict(
            title='% Plays',
            len=0.15,
            y=0.25
        ),
        cmin=0,
        # cmid=league_av_epa,
        colorscale=px.colors.sequential.Reds #px.colors.diverging.PRGn
    ),
    coloraxis2=dict(
        colorbar=dict(
            title='% Plays',
            len=0.15,
            y=0.05
        ),
        cmin=0,
        colorscale=px.colors.sequential.Reds
    ),
    legend=dict(
        x=1, y=1,
        title=dict(text='QB Position', font=dict(weight='bold')),
        traceorder='reversed',
    ),
    legend2=dict(
        x=1, y=0.75,
        title=dict(text='Play Type', font=dict(weight='bold')),
        traceorder='reversed',
    ),
    width=width, height=height,
    margin_pad=5,
    margin=dict(l=100, t=100, b=100)
)

fig.show()

# pio.write_image(fig, 'Tendencies.png', scale=6, width=width, height=height)

In [ ]:
print(pbp_matchup_teams.head().to_string())

In [ ]:
pass_tendencies_gpby = pbp_matchup_teams.loc[(pbp_matchup_teams['Offensive Snap']) & 
                                           (~pbp_matchup_teams['Is Special Teams Play']) &
                                           (pbp_matchup_teams['qtr'] <=3) &
                                           (pbp_matchup_teams['score_differential'] <= 10),:].groupby(['posteam', 'down', 'ydstogo']).aggregate(
                                                Plays=('Offensive Snap', 'sum'),
                                                Pass=('pass', 'sum'),
                                                Rush=('rush', 'sum'),
                                           ).rename_axis(index={'posteam': 'team'})


print(pass_tendencies_gpby.to_string())

In [ ]:
play_types = titans_o.groupby(['down', 'play_type']).aggregate(
    Plays=('play_type', 'size'),
    Yards=('yards_gained', 'sum')
)

print(play_types.to_string())